In [32]:
import game_2048
import pygame
import os
import numpy as np
import random
#import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

env =game_2048.GAME()

tf.reset_default_graph() # what does it really do?
#These lines establish the feed-forward part of the network used to choose actions
inputs1 = tf.placeholder(shape=[1,16],dtype=tf.float32) 
W = tf.Variable(tf.random_uniform([16,4],0,0.01))

Qout = tf.matmul(inputs1,W)

predict = tf.argmax(Qout,1)

#Below we obtain the loss by taking the sum of squares difference between the target and prediction Q values.
nextQ = tf.placeholder(shape=[16,4],dtype=tf.float32)
loss = tf.reduce_sum(tf.square(nextQ - Qout))
trainer = tf.train.GradientDescentOptimizer(learning_rate=0.1)
updateModel = trainer.minimize(loss)

init = tf.initialize_all_variables()
#saver = tf.train.Saver()

s = env.reset()
s # state at the begining - since this game has a number the state it is 0
#0, Left,1 Down, 2 Right, 3 Up

print("Ready to Train!")

Initialized GAME()
Ready to Train!


In [25]:
def plot_chart(y,n,plt):
    plt.figure()  # initialize plot
    axes = plt.gca()
    axes.set_xlim([0,n])
    axes.set_ylim([0,2048])
    plt.yticks(np.arange(0, 2048+128, 128))
    plt.xticks(np.arange(0, n+100, n/10))
    plt.ylabel('Game Score')
    plt.xlabel('Episodes')
    plt.suptitle('Learning Progression')
    plt.plot(y)
    plt.show()
    

In [33]:
# Set learning parameters
gamma = .99 # Gamma almost 1.0 - what does this mean? [0,1]
# higher gammma indicates the decision which gives the best reward is given bias (0.99 vs 0.99^2 (for next move and so on))
epsilon = 0.2   # epsilon value - How many times you want to use the predicted (learning NN) than a random input. 1 will mean never use a predicted value. 0 will mean always use the predicted value
# epsilon - selection of random.
num_episodes = 50#000 # a large number to stop runaway session (game)
#create lists to contain total rewards and steps per episode
envStepList = []  # steps in a particular game. i.e. while the game is running how many steps it went through.
rewardList = []  # reward list - keeping record of rewards on each episode 
with tf.Session() as sess:
    sess.run(init) # call the init function to initialize tensorflow variables.
    for episode in range(num_episodes):
        #save_path=saver.save(sess,os.path.join(os.getcwd(),'model','model_17052020.ckpt'))
        #Reset environment and get first new observation
        state = env.reset() # start a new game (game.refresh()) s should return state number integer e.g. 0
        rewardAll = 0        # reward All?
        dead = False       # dead is False when it is running
        envStep = 0          # step counter
        #The Q-Network
        while env.running:
#             pygame.event.pump()
            env.check_game_status() # returns True if game is win
#             env.score_board.run_number=i
            envStep+=1
            #Choose an action by greedily (with e chance of random action) from the Q-network
            action,allQ = sess.run([predict,Qout],feed_dict={inputs1:state.reshape(1,16)})
            print("--------------allQ-------------")
            print(allQ)
            if np.random.rand(1) < epsilon:
                action[0] = env.action_space_sample() # random movement from the game action space based on epsilon value
            #Get new state and reward from environment
            # env.step returns new state of the game, reward value (since minimization problem it could be 1/score)
            state1,reward,dead,_ = env.step(action[0]) # same as game.move(a) where a should be  randrange(4)
#             pygame.display.update()
            # capture image of the step
#             pygame.image.save(env.screen, os.path.join(os.getcwd(),'nn','episode_{:04d}_iter_{:04d}.jpg'.format(i,j)))
            
            #Obtain the Q' values by feeding the new state through our network
            Q1 = sess.run(Qout,feed_dict={inputs1:state1.reshape(1,16)})# get the next Q table Q(s1,a)
            print("--------------Q1-------------")
            print(Q1)
            #Obtain maxQ' and set our target value for chosen action.
            maxQ1 = np.max(Q1) # get the max Q-table value. for this random action 
            targetQ = allQ # target Q table - is zero at the begining of the run.
            targetQ[0,action[0]] = reward + gamma*maxQ1 # equation 1. storing long-term target memory for all the actons done till now.
            #Train our network using target and predicted Q values
            _,W1 = sess.run([updateModel,W],feed_dict={inputs1:state1.reshape(1,16),nextQ:targetQ}) # get the weights ignoring other outputs.
            rewardAll = max(rewardAll,reward) # total long-term reward. this means we will never reach zero.
            #print("Episode {}, reward value rAll = {}\nIs Game dead? {}".format(episode,rAll,d))
            state = state1 
            if dead == True:
                #print(env.game_state)
                #print("Episode {} died after {} steps".format(episode,envStep))
                #Reduce chance of random action as we train the model.
                epsilon = 1./((episode/50) + 10)
                break
            
        #print("{}\nEpisode {}\n Number of Moves = {}\n Maximum reward achieved={}".format("-"*50,i,j,rAll))
        envStepList.append(envStep)
        rewardList.append(rewardAll)
        if episode%100==0 or episode==num_episodes:
            plot_chart(rewardList,num_episodes,plt)
print("Maximum Score ever achieved: {}".format(max(rewardList)))

plt.plot(rewardList)

--------------allQ-------------
[[0.03033846 0.01675909 0.01457783 0.01803046]]
--------------Q1-------------
[[0.03165721 0.03586709 0.02432026 0.04699398]]


ValueError: Cannot feed value of shape (1, 4) for Tensor 'Placeholder_1:0', which has shape '(16, 4)'

In [ ]:
env.reset()


In [ ]:
env.running

In [ ]:
plt.plot(jList,rList,'.')

In [ ]:
import pygame
pygame.quit()

In [ ]:
pygame.quit()

In [ ]:
sess

In [ ]:
rList

In [30]:
Q1.shape

(1, 4)